## A. Configuración General.

In [1]:
#1. Librerías.
%run "../librerias.ipynb"

Matplotlib is building the font cache; this may take a moment.


/home/ianlink95/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/ianlink95/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#2. Constantes.
#a. Generales.
%run "../constantes.ipynb"

#b. A definir por el usuario.
dataset_undersampleado = dataset_file_fe_1_all_undersampleado

cantidad_meses_train = all
ventana = 1

mes_train = mes_train_all_menos_2_sin_rotas
mes_test = mes_test

storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = f"exp_lgbm_{cantidad_meses_train}_{ventana}_undersampling"

In [3]:
#3. Funciones
%run "../funciones.ipynb"

In [4]:
#4. Lectura de datos.
data_undersampleado = pd.read_parquet(dataset_undersampleado)

In [5]:
#5. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data_undersampleado['ctrx_quarter_normalizado'] = data_undersampleado['ctrx_quarter_normalizado'].astype(float)

#ii. Pesos.
data_undersampleado['clase_peso'] = 1.0

data_undersampleado.loc[data_undersampleado['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data_undersampleado.loc[data_undersampleado['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

#iii. Filtro Train.
data_undersampleado = data_undersampleado[data_undersampleado['foto_mes'].isin(mes_train)]

#iv. Separar en X e y después del undersampling para Optuna.
X_train_undersampleado = data_undersampleado.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria_undersampleado = data_undersampleado['clase_binaria']
w_train_undersampleado = data_undersampleado['clase_peso']

## B. Optimización Hiperparámetros (OH) con cantidad_meses_train meses con df -ventana con ratios incluidos.

In [6]:
#1. Funcion de optimización de hiperparámetros.
def objective(trial): 
    # Rango de parámetros a buscar sus valores óptimos.
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3) # mas bajo, más iteraciones necesita.
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 15, 900)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0)


    # Parámetros que le voy a pasar al modelo.
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    # Creo el dataset para Light GBM.
    train_data_ob = lgb.Dataset(X_train_undersampleado,
                              label=y_train_binaria_undersampleado, # eligir la clase
                              weight=w_train_undersampleado)
    
    # Entreno.
    cv_results = lgb.cv(
        params,
        train_data_ob,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior
        callbacks=[lgb.early_stopping(int(50 + 5 / learning_rate))],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    # Calculo la ganancia máxima y la mejor iteración donde se obtuvo dicha ganancia.
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5

In [7]:
#2. Voy a realizar un estudio de Optuna para encontrar los mejores parámetros.
#i. Creo la base de datos donde guardar los resultados.
storage_name = storage_name

study_name = study_name

#ii. Creo el estudio.
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

#iii. Corro el estudio.
study.optimize(objective, n_trials=100)

[I 2024-11-26 20:52:18,130] Using an existing study with name 'exp_lgbm_<built-in function all>_1_undersampling' instead of creating a new one.
[W 2024-11-26 20:52:36,124] Trial 2 failed with parameters: {'num_leaves': 183, 'learning_rate': 0.0448697711806425, 'min_data_in_leaf': 351, 'feature_fraction': 0.7106340382061375, 'bagging_fraction': 0.40764515117166755} because of the following error: ValueError('pandas dtypes must be int, float or bool.\nFields with bad pandas dtypes: mes_extraido: object').
Traceback (most recent call last):
  File "/home/ianlink95/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3156/2098973242.py", line 35, in objective
    cv_results = lgb.cv(
                 ^^^^^^^
  File "/home/ianlink95/.venv/lib/python3.12/site-packages/lightgbm/engine.py", line 774, in cv
    cvfolds = _make_n_folds(
              ^^^^^^^^^^^^^^
  File 

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: mes_extraido: object

In [10]:
#4. Visualizo los resultados del estudio, para modificar los rangos de análisis.

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'] )

In [ ]:
study.best_trial.params